<a href="https://colab.research.google.com/github/Ahmad162539/Smartbuilding/blob/main/University_Laboratory_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load data

In [1]:
import pandas as pd

url_univlab= 'https://raw.githubusercontent.com/Ahmad162539/Smartbuilding/main/data%20of%20UnivLab_Christy.csv'
univlab= pd.read_csv(url_univlab)
data_univlab= univlab[(univlab['timestamp'] > '2015-03-01') & (univlab['timestamp'] < '2015-06-01')]
dfs_univlab=data_univlab['energy']
datas_univlab=pd.DataFrame(dfs_univlab)


## import libraries

In [2]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

import numpy
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import metrics

import time

In [3]:
pip install EMD-signal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0


In [4]:
from PyEMD import CEEMDAN

In [5]:
pip install dataframe_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 32.1 MB/s eta 0:00:00


In [6]:
import dataframe_image as dfi

## Import all functions from another notebook for building prediction methods

In [7]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


#Parameters Settings

max_features=8 # the feature number of each node is set to 8, as it is suggested to be one-third of the feature's number (Lahouar and Slama,2017)
lr=0.001 #learning rate LSTM as recommended in (Kingma and Ba, 2014)
optimizer='Adam' #Adam is used as the optimizer for LSTM as it is computationally efficient (Dubey, Ashutosh Kumar, et al,2021)
neuron=64
epoch=100 #(Jorges et al, 2021)
batch_size=64 #(Kandel et al,2020)
hours=24 #in this study, to predict one hour ahead predicton, we use input from the previous 24 hour energy consumption
data_partition=0.8 #in this study, we divided the data into 80% of data as training data and 20% of the data as testing data

In [8]:
import numpy
import pandas as pd
import tensorflow as tf
from PyEMD import CEEMDAN
import warnings
warnings.filterwarnings("ignore")


### import the libraries
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from math import sqrt

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

def percentage_error(actual, predicted):
    res = numpy.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return res

def mean_absolute_percentage_error(y_true, y_pred):
    return numpy.mean(numpy.abs(percentage_error(numpy.asarray(y_true), numpy.asarray(y_pred)))) * 100



# In[25]:


def lr_model(datass,look_back,data_partition):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()
    import tensorflow as tf

    numpy.random.seed(1234)
    tf.random.set_seed(1234)

    from sklearn.linear_model import LinearRegression

    grid = LinearRegression()

    grid.fit(X,y)
    y_pred_train_lr= grid.predict(X)
    y_pred_test_lr= grid.predict(X1)

    y_pred_train_lr=pd.DataFrame(y_pred_train_lr)
    y_pred_test_lr=pd.DataFrame(y_pred_test_lr)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_lr= sc_y.inverse_transform (y_pred_test_lr)
    y_pred_train1_lr=sc_y.inverse_transform (y_pred_train_lr)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_rf=pd.DataFrame(y_pred_test1_lr)
    y_pred_train1_rf=pd.DataFrame(y_pred_train1_lr)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_lr)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_lr))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_lr)


    return mape,rmse,mae


# In[26]:


def svr_model(datass,look_back,data_partition):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    numpy.random.seed(1234)

    import tensorflow as tf
    tf.random.set_seed(1234)


    from sklearn.svm import SVR

    grid = SVR()
    grid.fit(X,y)
    y_pred_train_svr= grid.predict(X)
    y_pred_test_svr= grid.predict(X1)

    y_pred_train_svr=pd.DataFrame(y_pred_train_svr)
    y_pred_test_svr=pd.DataFrame(y_pred_test_svr)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_svr= sc_y.inverse_transform (y_pred_test_svr)
    y_pred_train1_svr=sc_y.inverse_transform (y_pred_train_svr)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_svr=pd.DataFrame(y_pred_test1_svr)
    y_pred_train1_svr=pd.DataFrame(y_pred_train1_svr)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_svr)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_svr))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_svr)


    return mape,rmse,mae


# In[27]:



def ann_model(datass,look_back,data_partition):


    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    import numpy

    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))


    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)


    from sklearn.neural_network import MLPRegressor

    model= MLPRegressor(random_state=1,activation='tanh').fit(X,y)

    numpy.random.seed(1234)


    # make predictions
    y_pred_train = model.predict(X)
    y_pred_test = model.predict(X1)
    y_pred_test= numpy.array(y_pred_test).ravel()

    y_pred_test=pd.DataFrame(y_pred_test)
    y1=pd.DataFrame(y1)

    y_pred_test1= sc_y.inverse_transform (y_pred_test)
    y_test= sc_y.inverse_transform (y1)







    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1)


    return mape,rmse,mae


# In[28]:


def rf_model(datass,look_back,data_partition,max_features):

    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    from sklearn.ensemble import RandomForestRegressor

    grid = RandomForestRegressor(max_features=max_features)
    grid.fit(X,y)
    y_pred_train_rf= grid.predict(X)
    y_pred_test_rf= grid.predict(X1)

    y_pred_train_rf=pd.DataFrame(y_pred_train_rf)
    y_pred_test_rf=pd.DataFrame(y_pred_test_rf)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)


    y_pred_test1_rf= sc_y.inverse_transform (y_pred_test_rf)
    y_pred_train1_rf=sc_y.inverse_transform (y_pred_train_rf)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)

    y_pred_test1_rf=pd.DataFrame(y_pred_test1_rf)
    y_pred_train1_rf=pd.DataFrame(y_pred_train1_rf)

    y_test= pd.DataFrame(y_test)


    #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,y_pred_test1_rf)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1_rf))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1_rf)


    return mape,rmse,mae


# In[29]:


def lstm_model(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):
    datasets=datass.values
    train_size = int(len(datasets) * data_partition)
    test_size = len(datasets) - train_size
    train, test = datasets[0:train_size], datasets[train_size:len(datasets)]
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)
    sc_X = StandardScaler()
    sc_y = StandardScaler()

    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()
    trainX1 = numpy.reshape(X, (X.shape[0],1,X.shape[1]))
    testX1 = numpy.reshape(X1, (X1.shape[0],1,X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    import os
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation
    from keras.layers import LSTM


    neuron=neuron
    model = Sequential()
    model.add(LSTM(units = neuron,input_shape=(trainX1.shape[1], trainX1.shape[2])))
    model.add(Dense(1))
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mse',optimizer=optimizer)
#    model.summary()


  # Fitting the RNN to the Training s
    model.fit(trainX1, y, epochs = epoch, batch_size = batch_size,verbose=0)
  # make predictions
    y_pred_train = model.predict(trainX1)
    y_pred_test = model.predict(testX1)
    y_pred_test= numpy.array(y_pred_test).ravel()

    y_pred_test=pd.DataFrame(y_pred_test)
    y_pred_test1= sc_y.inverse_transform (y_pred_test)
    y1=pd.DataFrame(y1)

    y_test= sc_y.inverse_transform (y1)

    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error

    from sklearn import metrics

    mape=mean_absolute_percentage_error(y_test,y_pred_test1)
    rmse= sqrt(mean_squared_error(y_test,y_pred_test1))
    mae=metrics.mean_absolute_error(y_test,y_pred_test1)
    return mape,rmse,mae


# In[30]:


###################################################hybrid based ceemdan####################################################
def hybrid_ceemdan_rf(datass,look_back,data_partition,max_features):


    import numpy as np
    import pandas as pd

    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T

    import pandas as pd

    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    for col in data_imf:

        datasetss2=pd.DataFrame(data_imf[col])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from sklearn.ensemble import RandomForestRegressor


        grid = RandomForestRegressor(max_features=max_features)
        grid.fit(X,y)
        y_pred_train= grid.predict(X)
        y_pred_test= grid.predict(X1)

        y_pred_test=pd.DataFrame(y_pred_test)
        y_pred_train=pd.DataFrame(y_pred_train)

        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)

        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)


    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)
    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)




    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)




   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)



    return mape,rmse,mae


# In[31]:


def hybrid_ceemdan_lstm(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):


    from PyEMD import CEEMDAN

    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T

    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    for col in data_imf:

        datasetss2=pd.DataFrame(data_imf[col])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from keras.models import Sequential
        from keras.layers.core import Dense, Dropout, Activation
        from keras.layers import LSTM


        neuron=neuron

        model = Sequential()
        model.add(LSTM(units = neuron,input_shape=(trainX.shape[1], trainX.shape[2])))
        model.add(Dense(1))
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mse',optimizer=optimizer)

        numpy.random.seed(1234)


        # Fitting the RNN to the Training set
        model.fit(trainX, y, epochs = epoch, batch_size = batch_size,verbose=0)

        # make predictions
        y_pred_train = model.predict(trainX)
        y_pred_test = model.predict(testX)

        # make predictions

        y_pred_test= numpy.array(y_pred_test).ravel()
        y_pred_test=pd.DataFrame(y_pred_test)
        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)
        y_pred_train= numpy.array(y_pred_train).ravel()
        y_pred_train=pd.DataFrame(y_pred_train)


        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)


    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)

    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)


    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)



   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)

    return mape,rmse,mae


# In[32]:


def proposed_method(datass,look_back,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer):

    from PyEMD import CEEMDAN


    dfs=datass
    s = dfs.values

    emd = CEEMDAN(epsilon=0.05)
    emd.noise_seed(12345)

    IMFs = emd(s)


    full_imf=pd.DataFrame(IMFs)
    data_imf=full_imf.T



    pred_test=[]
    test_ori=[]
    pred_train=[]
    train_ori=[]


    n_imf=len(data_imf.columns)

    k=list(range(1,n_imf))
    m=[0]


    for i in m:

        datasetss2=pd.DataFrame(data_imf[i])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from sklearn.ensemble import RandomForestRegressor


        grid = RandomForestRegressor(max_features=max_features)
        grid.fit(X,y)
        y_pred_train= grid.predict(X)
        y_pred_test= grid.predict(X1)

        y_pred_test=pd.DataFrame(y_pred_test)
        y_pred_train=pd.DataFrame(y_pred_train)

        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)

        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)




    for i in k:

        datasetss2=pd.DataFrame(data_imf[i])
        datasets=datasetss2.values
        train_size = int(len(datasets) * data_partition)
        test_size = len(datasets) - train_size
        train, test = datasets[0:train_size], datasets[train_size:len(datasets)]

        trainX, trainY = create_dataset(train, look_back)
        testX, testY = create_dataset(test, look_back)
        X_train=pd.DataFrame(trainX)
        Y_train=pd.DataFrame(trainY)
        X_test=pd.DataFrame(testX)
        Y_test=pd.DataFrame(testY)
        sc_X = StandardScaler()
        sc_y = StandardScaler()
        X= sc_X.fit_transform(X_train)
        y= sc_y.fit_transform(Y_train)
        X1= sc_X.fit_transform(X_test)
        y1= sc_y.fit_transform(Y_test)
        y=y.ravel()
        y1=y1.ravel()

        import numpy

        trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
        testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

        numpy.random.seed(1234)
        import tensorflow as tf
        tf.random.set_seed(1234)


        from keras.models import Sequential
        from keras.layers.core import Dense, Dropout, Activation
        from keras.layers import LSTM

        neuron=neuron

        model = Sequential()
        model.add(LSTM(units = neuron,input_shape=(trainX.shape[1], trainX.shape[2])))
        model.add(Dense(1))
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mse',optimizer=optimizer)


        numpy.random.seed(1234)


     # Fitting the RNN to the Training set
        model.fit(trainX, y, epochs = epoch, batch_size = batch_size,verbose=0)


        # make predictions
        y_pred_train = model.predict(trainX)
        y_pred_test = model.predict(testX)

        # make predictions


        y_pred_test= numpy.array(y_pred_test).ravel()
        y_pred_test=pd.DataFrame(y_pred_test)
        y1=pd.DataFrame(y1)
        y=pd.DataFrame(y)
        y_pred_train= numpy.array(y_pred_train).ravel()
        y_pred_train=pd.DataFrame(y_pred_train)


        y_test= sc_y.inverse_transform (y1)
        y_train= sc_y.inverse_transform (y)

        y_pred_test1= sc_y.inverse_transform (y_pred_test)
        y_pred_train1= sc_y.inverse_transform (y_pred_train)


        pred_test.append(y_pred_test1)
        test_ori.append(y_test)
        pred_train.append(y_pred_train1)
        train_ori.append(y_train)

    result_pred_test= pd.DataFrame.from_records(pred_test)
    result_pred_train= pd.DataFrame.from_records(pred_train)


    a=result_pred_test.sum(axis = 0, skipna = True)
    b=result_pred_train.sum(axis = 0, skipna = True)


    dataframe=pd.DataFrame(dfs)
    dataset=dataframe.values

    train_size = int(len(dataset) * data_partition)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size], dataset[train_size:len(dataset)]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    X_train=pd.DataFrame(trainX)
    Y_train=pd.DataFrame(trainY)
    X_test=pd.DataFrame(testX)
    Y_test=pd.DataFrame(testY)

    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X= sc_X.fit_transform(X_train)
    y= sc_y.fit_transform(Y_train)
    X1= sc_X.fit_transform(X_test)
    y1= sc_y.fit_transform(Y_test)
    y=y.ravel()
    y1=y1.ravel()


    trainX = numpy.reshape(X, (X.shape[0], 1, X.shape[1]))
    testX = numpy.reshape(X1, (X1.shape[0], 1, X1.shape[1]))

    numpy.random.seed(1234)
    import tensorflow as tf
    tf.random.set_seed(1234)

    y1=pd.DataFrame(y1)
    y=pd.DataFrame(y)
    y_test= sc_y.inverse_transform (y1)
    y_train= sc_y.inverse_transform (y)


    a= pd.DataFrame(a)
    y_test= pd.DataFrame(y_test)


    import numpy as np


   #summarize the fit of the model
    mape=mean_absolute_percentage_error(y_test,a)
    rmse= sqrt(mean_squared_error(y_test,a))
    mae=metrics.mean_absolute_error(y_test,a)

    return mape,rmse,mae,a,y_test


## Import parameter settings

## Run the experiments
### Run this following cell will train and test the proposed method and other benchmark methods on University Laboratory Dataset

In [9]:
#Linear Regression

start_time = time.time()
lr_univlab=lr_model(datas_univlab,hours,data_partition)
lr_time_univlab=time.time() - start_time
print("--- %s seconds - Linear Regression- univlab ---" % (lr_time_univlab))

#Support Vector Regression
start_time = time.time()
svr_univlab=svr_model(datas_univlab,hours,data_partition)
svr_time_univlab=time.time() - start_time
print("--- %s seconds - Support Vector Regression- univlab ---" % (svr_time_univlab))


#ANN
start_time = time.time()
ann_univlab=ann_model(datas_univlab,hours,data_partition)
ann_time_univlab=time.time() - start_time
print("--- %s seconds - ANN- univlab ---" % (ann_time_univlab))

#random forest
start_time = time.time()
rf_univlab=rf_model(datas_univlab,hours,data_partition,max_features)
rf_time_univlab=time.time() - start_time
print("--- %s seconds - Random Forest- univlab ---" % (rf_time_univlab))

#LSTM
start_time = time.time()
lstm_univlab=lstm_model(datas_univlab,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
lstm_time_univlab=time.time() - start_time
print("--- %s seconds - lstm- univlab ---" % (lstm_time_univlab))


#CEEMDAN RF
start_time = time.time()
ceemdan_rf_univlab=hybrid_ceemdan_rf(dfs_univlab,hours,data_partition,max_features)
ceemdan_rf_time_univlab=time.time() - start_time
print("--- %s seconds - ceemdan_rf- univlab ---" % (ceemdan_rf_time_univlab))

#CEEMDAN LSTM
start_time = time.time()
ceemdan_lstm_univlab=hybrid_ceemdan_lstm(dfs_univlab,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
ceemdan_lstm_time_univlab=time.time() - start_time
print("--- %s seconds - ceemdan_lstm- univlab ---" % (ceemdan_lstm_time_univlab))


#proposed method
start_time = time.time()
proposed_method_univlab=proposed_method(dfs_univlab,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
proposed_method_time_univlab=time.time() - start_time
print("--- %s seconds - proposed_method- univlab ---" % (proposed_method_time_univlab))



--- 0.3701629638671875 seconds - Linear Regression- univlab ---
--- 1.1945862770080566 seconds - Support Vector Regression- univlab ---
--- 15.744020462036133 seconds - ANN- univlab ---
--- 2.368340492248535 seconds - Random Forest- univlab ---
13/13 [==============================] - 0s 3ms/step
--- 17.50617289543152 seconds - lstm- univlab ---
--- 31.127997636795044 seconds - ceemdan_rf- univlab ---
13/13 [==============================] - 0s 2ms/step
--- 118.58984112739563 seconds - ceemdan_lstm- univlab ---
13/13 [==============================] - 0s 3ms/step
--- 108.96115708351135 seconds - proposed_method- univlab ---


## Summarize of experimental results with running time
### Run this following cell will summarize the result and generate output used in Section 4.4 (Table 3) for University Laboratory dataset

In [10]:
running_time_univlab=pd.DataFrame([lr_time_univlab,svr_time_univlab,ann_time_univlab,
                                   rf_time_univlab,lstm_time_univlab,ceemdan_rf_time_univlab,
                                   ceemdan_lstm_time_univlab,proposed_method_time_univlab])
running_time_univlab=running_time_univlab.T
running_time_univlab.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']

proposed_method_univlab_df=proposed_method_univlab[0:3]
result_univlab=pd.DataFrame([lr_univlab,svr_univlab,ann_univlab,rf_univlab,lstm_univlab,ceemdan_rf_univlab,
                    ceemdan_lstm_univlab,proposed_method_univlab_df])
result_univlab=result_univlab.T
result_univlab.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']
univlab_summary=pd.concat([result_univlab,running_time_univlab],axis=0)

univlab_summary.set_axis(['MAPE(%)', 'RMSE','MAE','running time (s)'], axis='index')

univlab_summary.style.set_caption("University Laboratory Results")
index = univlab_summary.index
index.name = "university laboratory results"
univlab_summary

,LR,SVR,ANN,RF,LSTM,CEEMDAN RF,CEEMDAN LSTM,Proposed Method
university laboratory results,,,,,,,,
0,5.858858,6.419207,6.134858,6.300708,6.464001,3.526199,3.575848,3.140996
1,2.579903,2.706899,2.658628,2.728669,2.694062,1.461045,1.432516,1.268738
2,1.864822,2.093199,2.004848,2.057561,2.085680,1.118182,1.135480,0.999068
0,0.370163,1.194586,15.744020,2.368340,17.506173,31.127998,118.589841,108.961157


In [ ]:
#export table to png
#dfi.export(univlab_summary,"univlab_summary_table.png")

## Calculate percentage improvement
### Run this following cell will calculate percentage improvement and generate output used in Section 4.4 (Table 4) for University Laboratory dataset

In [11]:
pMAPE_LR_vs_Proposed_univlab=((lr_univlab[0]-proposed_method_univlab[0])/lr_univlab[0])*100
pRMSE_LR_vs_Proposed_univlab=((lr_univlab[1]-proposed_method_univlab[1])/lr_univlab[1])*100
pMAE_LR_vs_Proposed_univlab=((lr_univlab[2]-proposed_method_univlab[2])/lr_univlab[2])*100

pMAPE_SVR_vs_Proposed_univlab=((svr_univlab[0]-proposed_method_univlab[0])/svr_univlab[0])*100
pRMSE_SVR_vs_Proposed_univlab=((svr_univlab[1]-proposed_method_univlab[1])/svr_univlab[1])*100
pMAE_SVR_vs_Proposed_univlab=((svr_univlab[2]-proposed_method_univlab[2])/svr_univlab[2])*100

pMAPE_ANN_vs_Proposed_univlab=((ann_univlab[0]-proposed_method_univlab[0])/ann_univlab[0])*100
pRMSE_ANN_vs_Proposed_univlab=((ann_univlab[1]-proposed_method_univlab[1])/ann_univlab[1])*100
pMAE_ANN_vs_Proposed_univlab=((ann_univlab[2]-proposed_method_univlab[2])/ann_univlab[2])*100

pMAPE_RF_vs_Proposed_univlab=((rf_univlab[0]-proposed_method_univlab[0])/rf_univlab[0])*100
pRMSE_RF_vs_Proposed_univlab=((rf_univlab[1]-proposed_method_univlab[1])/rf_univlab[1])*100
pMAE_RF_vs_Proposed_univlab=((rf_univlab[2]-proposed_method_univlab[2])/rf_univlab[2])*100

pMAPE_LSTM_vs_Proposed_univlab=((lstm_univlab[0]-proposed_method_univlab[0])/lstm_univlab[0])*100
pRMSE_LSTM_vs_Proposed_univlab=((lstm_univlab[1]-proposed_method_univlab[1])/lstm_univlab[1])*100
pMAE_LSTM_vs_Proposed_univlab=((lstm_univlab[2]-proposed_method_univlab[2])/lstm_univlab[2])*100

pMAPE_ceemdan_rf_vs_Proposed_univlab=((ceemdan_rf_univlab[0]-proposed_method_univlab[0])/ceemdan_rf_univlab[0])*100
pRMSE_ceemdan_rf_vs_Proposed_univlab=((ceemdan_rf_univlab[1]-proposed_method_univlab[1])/ceemdan_rf_univlab[1])*100
pMAE_ceemdan_rf_vs_Proposed_univlab=((ceemdan_rf_univlab[2]-proposed_method_univlab[2])/ceemdan_rf_univlab[2])*100


pMAPE_ceemdan_lstm_vs_Proposed_univlab=((ceemdan_lstm_univlab[0]-proposed_method_univlab[0])/ceemdan_lstm_univlab[0])*100
pRMSE_ceemdan_lstm_vs_Proposed_univlab=((ceemdan_lstm_univlab[1]-proposed_method_univlab[1])/ceemdan_lstm_univlab[1])*100
pMAE_ceemdan_lstm_vs_Proposed_univlab=((ceemdan_lstm_univlab[2]-proposed_method_univlab[2])/ceemdan_lstm_univlab[2])*100


df_PI_univlab=[[pMAPE_LR_vs_Proposed_univlab,pMAPE_SVR_vs_Proposed_univlab,pMAPE_ANN_vs_Proposed_univlab,
                pMAPE_RF_vs_Proposed_univlab,pMAPE_LSTM_vs_Proposed_univlab,pMAPE_ceemdan_rf_vs_Proposed_univlab,
                pMAPE_ceemdan_lstm_vs_Proposed_univlab],
                [pRMSE_LR_vs_Proposed_univlab,pRMSE_SVR_vs_Proposed_univlab,pRMSE_ANN_vs_Proposed_univlab,
                pRMSE_RF_vs_Proposed_univlab,pRMSE_LSTM_vs_Proposed_univlab,pRMSE_ceemdan_rf_vs_Proposed_univlab,
                pRMSE_ceemdan_lstm_vs_Proposed_univlab],
                [pMAE_LR_vs_Proposed_univlab,pMAE_SVR_vs_Proposed_univlab,pMAE_ANN_vs_Proposed_univlab,
                pMAE_RF_vs_Proposed_univlab,pMAE_LSTM_vs_Proposed_univlab,pMAE_ceemdan_rf_vs_Proposed_univlab,
                pMAE_ceemdan_lstm_vs_Proposed_univlab]]

PI_univlab=pd.DataFrame(df_PI_univlab, columns=["Proposed Method vs.LR", "Proposed Method vs.SVR"," Proposed Method vs.ANN",
                                      "Proposed Method vs.RF","Proposed Method vs.LSTM","Proposed Method vs.CEEMDAN RF",
                                      "Proposed Method vs. CEEMDAN LSTM"])
PI_univlab= PI_univlab.round(decimals = 2)
PI_univlab.set_axis(['MAPE(%)', 'RMSE','MAE'], axis='index')
PI_univlab.style.set_caption("Percentage Improvement-University Laboratory Building")
index = PI_univlab.index
index.name = "Percentage Improvement university laboratory"
PI_univlab

,Proposed Method vs.LR,Proposed Method vs.SVR,Proposed Method vs.ANN,Proposed Method vs.RF,Proposed Method vs.LSTM,Proposed Method vs.CEEMDAN RF,Proposed Method vs. CEEMDAN LSTM
Percentage Improvement university laboratory,,,,,,,
0,46.39,51.07,48.80,50.15,51.41,10.92,12.16
1,50.82,53.13,52.28,53.50,52.91,13.16,11.43
2,46.43,52.27,50.17,51.44,52.10,10.65,12.01


In [ ]:
#export table to png
#dfi.export(PI_univlab,"PI_univlab_table.png")